In [1]:
import openturns as ot
import numpy as np
import otbenchmark as otb
import pandas as pd

In [2]:
p = otb.OTReliabilityAlgorithmBenchmark()
list = p.problemslist

In [3]:
k = 17

In [4]:
name = list[k].getName()
name

'RP110'

In [5]:
N = 10 ** 9
N

1000000000

In [6]:
name = list[k].getName()

resultsFORM = otb.OTReliabilityAlgorithmBenchmark.FORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsSORM = otb.OTReliabilityAlgorithmBenchmark.SORM(
        list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3)

resultsMC = otb.OTReliabilityAlgorithmBenchmark.MonteCarloSampling(list[k], maximumOuterSampling=N,
                                                                   coefficientOfVariation=0.1, blockSize=2)

resultFORMIS = otb.OTReliabilityAlgorithmBenchmark.FORMImportanceSampling(list[k],
        nearestPointAlgo="Cobyla",
        maximumEvaluationNumber=100,
        maximumAbsoluteError=1.0e-3,
        maximumRelativeError=1.0e-3,
        maximumResidualError=1.0e-3,
        maximumConstraintError=1.0e-3,
        maximumOuterSampling=N,
        coefficientOfVariation=0.1)

resultSubset = otb.OTReliabilityAlgorithmBenchmark.SubsetSampling(
        list[k], maximumOuterSampling=10000, coefficientOfVariation=0.1, blockSize=1)

result = [resultsMC, resultsFORM, resultsSORM, resultFORMIS, resultSubset]

In [7]:
a = "[%.5f, %.5f] " % ( resultsMC[4], resultsMC[5])


b = "[%.5f, %.5f] " % ( resultFORMIS[4], resultFORMIS[5])


c = "[%.5f, %.5f] " % ( resultSubset[4], resultSubset[5])


In [8]:
mc1 = np.format_float_scientific(np.float32(resultsMC[4]))
mc2 = np.format_float_scientific(np.float32(resultsMC[5]))

ic1 = "["+mc1+", "+mc2+"]"
ic1

'[2.4884861e-05, 3.7017344e-05]'

In [9]:
formis1 = np.format_float_scientific(np.float32(resultFORMIS[4]))
formis2 = np.format_float_scientific(np.float32(resultFORMIS[5]))

ic2 = "["+formis1+", "+formis2+"]"
ic2

'[2.4015671e-05, 3.5635116e-05]'

In [10]:
subset1 = np.format_float_scientific(np.float32(resultSubset[4]))
subset2 = np.format_float_scientific(np.float32(resultSubset[5]))

ic3 = "["+subset1+", "+subset2+"]"

ic3

'[1.6653868e-05, 2.7150343e-05]'

In [11]:
M = [[None]*5]*6

M[2][3] = a

M

[[None, None, None, '[0.00002, 0.00004] ', None],
 [None, None, None, '[0.00002, 0.00004] ', None],
 [None, None, None, '[0.00002, 0.00004] ', None],
 [None, None, None, '[0.00002, 0.00004] ', None],
 [None, None, None, '[0.00002, 0.00004] ', None],
 [None, None, None, '[0.00002, 0.00004] ', None]]

In [12]:
l = np.array([[1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a], [1, 1, 1, ic3, 1, a],[1, 1, 1, ic3, 1, a]])

In [13]:

method = ["CMC", "FORM", "SORM", "FORM-IS", "SUBSET"]
metrics = ["Probabilité de référence de "+name, "Probabilité estimée", "Erreur absolue","Intervalle de confiance",
           "nombre d'évaluations", "Log erreur relative"]

results = l



for i in range(len(method)):
    results[i][0] =  np.format_float_scientific(np.float32(list[k].getProbability()))

for i in range(len(method)):
    results[i][1] =  np.format_float_scientific(np.float32(result[i][0]))

for i in range(len(method)):
    results[i][2] =  np.format_float_scientific(np.float32(result[i][1]))

for i in range(len(method)):
    results[i][4] =  result[i][3]
    
for i in range(len(method)):
    results[i][5] =  "%.2f " % (result[i][2])


results[0][3] = ic1
results[1][3] = "-----------------"
results[2][3] = "-----------------"
results[3][3] = ic2
results[4][3] = ic3


df = pd.DataFrame(results, index = method, columns=metrics)
print("Function g : ", list[k].getEvent().getFunction())
df

Function g :  [x0,x1]->[if (x0 <= 3.5)
var g1 := 0.85 - 0.1 * x0;
else
  g1 := 4 - x0;
if (x1 <= 2.0)
  var g2 := 2.3 - x1;
else
  g2 := 0.5 - 0.1 * x1;
gsys := min(g1, g2);]


,Probabilité de référence de RP110,Probabilité estimée,Erreur absolue,Intervalle de confiance,nombre d'évaluations,Log erreur relative
CMC,3.1e-05,3.09511e-05,4.8897377e-08,"[2.4884861e-05, 3.7017344e-05]",4846354,2.80
FORM,3.1e-05,3.18053e-05,8.052984e-07,-----------------,104,1.59
SORM,3.1e-05,3.18053e-05,8.052984e-07,-----------------,117,1.59
FORM-IS,3.1e-05,2.9825394e-05,1.1746063e-06,"[2.4015671e-05, 3.5635116e-05]",547,1.42
SUBSET,3.1e-05,2.1902106e-05,9.097894e-06,"[1.6653868e-05, 2.7150343e-05]",50000,0.53


In [14]:
df.to_csv('problem110.csv')